# Preliminaries

In [ ]:
# Install Reka API.
!python3 -m pip install --quiet reka-api

In [ ]:
from IPython.display import Image
import os
import reka
import time

from getpass import getpass

# Assuming this is the IP of a VM with on-prem deployment
# reka._SERVER = "http://***********:8000"
# reka.API_KEY = "on_prem_key"

# To use the prod server:
reka._SERVER = "https://api.reka.ai"
reka.API_KEY = getpass("Enter your Reka API key: ")

Enter your Reka API key: ··········


# Chat capabilities

In [ ]:
# Very simple chat interface.
response = reka.chat("What is the capital of the UK?")
print(response["text"])

The capital of the UK is London.


In [ ]:
# You can also specify advanced parameters to control generation.
response = reka.chat(
    "List some large cities in the UK.",
    assistant_start_text="1.",
    request_output_len=128,
    temperature=0.4,
    stop_tokens=["11."],
)
print(response["text"])

1. London

2. Birmingham

3. Manchester

4. Leeds

5. Sheffield

6. Glasgow

7. Edinburgh

8. Liverpool

9. Bristol

10. Cardiff


# Retrieval capabilities

In [ ]:
# Download a toy dataset.
!wget https://reka-public.s3.eu-west-2.amazonaws.com/amazon_legal_docs.zip

--2023-09-12 13:55:12--  https://reka-public.s3.eu-west-2.amazonaws.com/amazon_legal_docs.zip
Resolving reka-public.s3.eu-west-2.amazonaws.com (reka-public.s3.eu-west-2.amazonaws.com)... 52.95.144.26, 52.95.149.162, 52.95.150.10, ...
Connecting to reka-public.s3.eu-west-2.amazonaws.com (reka-public.s3.eu-west-2.amazonaws.com)|52.95.144.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155322 (152K) [application/zip]
Saving to: ‘amazon_legal_docs.zip’

amazon_legal_docs.z 100%[===================>] 151.68K   509KB/s    in 0.3s    

2023-09-12 13:55:13 (509 KB/s) - ‘amazon_legal_docs.zip’ saved [155322/155322]



In [ ]:
reka.add_dataset(
    filepath="amazon_legal_docs.zip",
    name="amazon",
)

{'name': 'amazon', 'ok': True, 'info': ''}

In [ ]:
# Prepare the documents for retrieval.
job_id = reka.prepare_retrieval(dataset_name="amazon")

while True:
    status = reka.retrieval_job_status(job_id)
    if status.is_done():
        break
    time.sleep(5)

print(f"Job finished with status: {status.job_status.name}")

Job finished with status: COMPLETE


In [ ]:
# The model now knows everything about Amazon legal documentation.
response = reka.chat(
    "What is the refund policy for Amazon Prime memberships that were redeemed through a promotional or gift code?",
    retrieval_dataset="amazon",
    temperature=0.3,
)
print(response["text"])

According to the Amazon Prime Terms and Conditions, Prime memberships redeemed through a promotional or gift code are not refundable. This means that if you paid for a Prime membership using a promotional or gift code, you will not be able to receive a refund for that membership, even if you have not used any of the Prime benefits.

If you have any questions or concerns about your Amazon Prime membership, you can contact Amazon's customer service for assistance.


In [ ]:
# Cleanup
reka.delete_dataset("amazon")

# Image understanding

In [ ]:
# Let's use an image of a cat.
image_url = "https://docs.reka.ai/_images/000000245576.jpg"
Image(url=image_url)

In [ ]:
# Ask the model what's in the picture.
response = reka.chat(
    "What's in the photo?",
    media_url="https://docs.reka.ai/_images/000000245576.jpg",
    media_type="image",
    model_name="default_mmlm",
)
print(response["text"])

There is an orange cat that is sitting in front of a computer keyboard. The cat is seen licking a keyboard key.


In [ ]:
# Continue the conversation.
conversation_history = [{
        "type": "human",
        "text": "What's in the photo?",
        "media_url": "https://docs.reka.ai/_images/000000245576.jpg",
        "media_type": "image",
    },{
        "type": "model",
        "text": response["text"],
    }]
response = reka.chat("What color is the keyboard?", conversation_history=conversation_history, model_name="default_mmlm")
print(response['text'])

The color of the keyboard is white.
